# Data Cleaning and Preprocessing Notebook

This notebook is designed to clean and preprocess data from various sources, including news articles, fake news datasets, and stock market data. The workflow includes loading raw data, cleaning it, extracting relevant information, and saving the processed data for further analysis. 

### Key Sections:
1. **News Data**: 
    - Extract and clean data from sources like Alpha Vantage, Markets Insider, and Google Search.
    - Merge and deduplicate news data for multiple stock symbols.

2. **Fake News Data**:
    - Process training and testing datasets for fake news detection.
    - Remove duplicates and save cleaned datasets.

3. **Stock Data**:
    - Clean and preprocess stock market data from Alpha Vantage.
    - Standardize column names and formats.

This notebook ensures that the data is ready for downstream tasks such as analysis, modeling, and visualization.

In [1]:
# Standard libraries
import pandas as pd
import os

# Enable auto-reload for modules during development
%load_ext autoreload
%autoreload 2

# Set display options for Pandas to show all columns
pd.set_option('display.max_columns', None)

# Custom libraries
from nlp_scripts import data_cleaning as cl
from nlp_scripts import data_check as ck

In [2]:
# Data folder path
data_folder = '../data/data_to_clean/'

# Data folder path cleaned data
data_folder_cleaned = '../data/data_cleaned'

# Load news dataframes
news_dict = cl.load_news_dataframes(data_folder)

# Load stocks dataframes from Alpha Vantage
stocks_dict = cl.load_stocks_dataframes(data_folder, 'stocks', 'alpha_vantage.csv')
# Load stocks dataframes from Yahoo Finance
stocks_dict_yahoo = cl.load_stocks_dataframes(data_folder, 'stock', 'yahoo.csv')

In [47]:
# The keys that define the different data frames in news_dict
news_dict.keys()

dict_keys(['AAPL_alpha_vantage', 'AAPL_markets_insider', 'AMZN_google_search', 'AMZN_alpha_vantage', 'AMZN_markets_insider', 'AAPL_google_search', 'BAC_alpha_vantage', 'BAC_markets_insider', 'BAC_google_search', 'GME_google_search', 'GME_alpha_vantage', 'GME_markets_insider', 'GS_google_search', 'GS_alpha_vantage', 'GS_markets_insider', 'NVDA_alpha_vantage', 'NVDA_markets_insider', 'NVDA_google_search', 'TSLA_google_search', 'TSLA_alpha_vantage', 'TSLA_markets_insider'])

In [48]:
# The keys that define the different data frames in stocks_dict
stocks_dict.keys()

dict_keys(['AAPL_alpha_vantage', 'AMZN_alpha_vantage', 'BAC_alpha_vantage', 'GME_alpha_vantage', 'GS_alpha_vantage', 'NVDA_alpha_vantage', 'TSLA_alpha_vantage'])

In [49]:
# The keys that define the different data frames in stocks_dict_yahoo
stocks_dict_yahoo.keys()

dict_keys(['stock_AAPL_yahoo', 'stock_AMZN_yahoo', 'stock_BAC_yahoo', 'stock_GME_yahoo', 'stock_GS_yahoo', 'stock_NVDA_yahoo', 'stock_TSLA_yahoo'])

## News

### Alpha Vantage

In [3]:
# Let's check the columns for the AAPL_alpha_vantage dataframe
news_dict['AAPL_alpha_vantage'].head(2)

,title,url,time_published,authors,summary,banner_image,source,category_within_source,source_domain,topics,overall_sentiment_score,overall_sentiment_label,ticker_sentiment
0,"Trump's Tariffs Stir Global Markets, Wall Stre...",https://www.benzinga.com/media/25/04/44657800/...,2025-04-06 10:00:38,['Ananya Gairola'],The past week was a rollercoaster ride for glo...,https://cdn.benzinga.com/files/images/story/20...,Benzinga,News,www.benzinga.com,"[{'topic': 'Economy - Monetary', 'relevance_sc...",0.002465,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.2369..."
1,3 Top Buffett Stocks to Buy and Hold for the N...,https://www.fool.com/investing/2025/04/05/3-to...,2025-04-05 22:03:00,"['John Ballard', 'and Jennifer Saibil', 'Jerem...",Warren Buffett has built a fortune for Berkshi...,https://g.foolcdn.com/editorial/images/813433/...,Motley Fool,NaN,www.fool.com,"[{'topic': 'Retail & Wholesale', 'relevance_sc...",0.394156,Bullish,"[{'ticker': 'DMPZF', 'relevance_score': '0.129..."


In [4]:
# Extracting data from Alpha Vantage
# This function will extract data from Alpha Vantage and return two dataframes:
# one with title, date and scores and another with minimal with date and title only.
full_data_by_stock, minimal_data_by_stock = cl.extract_alpha_vantage_per_stock(news_dict)

In [52]:
# Lets check the dataframes keys
full_data_by_stock.keys()

dict_keys(['AAPL', 'AMZN', 'BAC', 'GME', 'GS', 'NVDA', 'TSLA'])

In [53]:
# Lets check the columns for the AAPL_alpha_vantage dataframe with the sentiment scores
full_data_by_stock['AAPL'].head(2)

,title,date,summary,overall_sentiment_score,overall_sentiment_label,ticker_sentiment,source
0,"Trump's Tariffs Stir Global Markets, Wall Stre...",2025-04-06 10:00:38,The past week was a rollercoaster ride for glo...,0.002465,Neutral,"[{'ticker': 'AAPL', 'relevance_score': '0.2369...",Alpha Vantage
1,3 Top Buffett Stocks to Buy and Hold for the N...,2025-04-05 22:03:00,Warren Buffett has built a fortune for Berkshi...,0.394156,Bullish,"[{'ticker': 'DMPZF', 'relevance_score': '0.129...",Alpha Vantage


In [54]:
# Lets check the columns for the AAPL_alpha_vantage dataframe without the sentiment scores
minimal_data_by_stock['AAPL'].head(2)

,title,date,source
0,"Trump's Tariffs Stir Global Markets, Wall Stre...",2025-04-06 10:00:38,Alpha Vantage
1,3 Top Buffett Stocks to Buy and Hold for the N...,2025-04-05 22:03:00,Alpha Vantage


In [5]:
# Set date as index and format columns 
dataframes_cleaned_alpha_vantage = cl.convert_date(minimal_data_by_stock)
dataframes_cleaned_alpha_vantage_with_sentiment = cl.convert_date(full_data_by_stock)

Stock: AAPL
First date: 2025-02-12
Last date: 2025-04-06
------------------------------
Stock: AMZN
First date: 2025-02-06
Last date: 2025-04-06
------------------------------
Stock: BAC
First date: 2025-01-28
Last date: 2025-04-06
------------------------------
Stock: GME
First date: 2024-06-10
Last date: 2025-04-04
------------------------------
Stock: GS
First date: 2025-02-21
Last date: 2025-04-06
------------------------------
Stock: NVDA
First date: 2025-03-20
Last date: 2025-04-06
------------------------------
Stock: TSLA
First date: 2025-03-15
Last date: 2025-04-06
------------------------------
Stock: AAPL
First date: 2025-02-12
Last date: 2025-04-06
------------------------------
Stock: AMZN
First date: 2025-02-06
Last date: 2025-04-06
------------------------------
Stock: BAC
First date: 2025-01-28
Last date: 2025-04-06
------------------------------
Stock: GME
First date: 2024-06-10
Last date: 2025-04-04
------------------------------
Stock: GS
First date: 2025-02-21
Last 

In [6]:
# Moving date column as first column
dataframes_cleaned_alpha_vantage = cl.move_date_column_to_front(dataframes_cleaned_alpha_vantage)

In [57]:
# Lets check the columns for the AAPL_alpha_vantage dataframe without the sentiment scores
dataframes_cleaned_alpha_vantage['AAPL'].head(2)

,date,title,source
0,2025-04-06,"Trump's Tariffs Stir Global Markets, Wall Stre...",Alpha Vantage
1,2025-04-05,3 Top Buffett Stocks to Buy and Hold for the N...,Alpha Vantage


In [58]:
# Saving the data frames
cl.save_cleaned_dataframes(dataframes_cleaned_alpha_vantage, "alpha_vantage", data_folder_cleaned, prefix_word="news")
cl.save_cleaned_dataframes(dataframes_cleaned_alpha_vantage_with_sentiment, "alpha_vantage_with_sentiment", data_folder_cleaned, prefix_word="news")

Saved news_cleaned_dataframe_AAPL_alpha_vantage.csv
Saved news_cleaned_dataframe_AMZN_alpha_vantage.csv
Saved news_cleaned_dataframe_BAC_alpha_vantage.csv
Saved news_cleaned_dataframe_GME_alpha_vantage.csv
Saved news_cleaned_dataframe_GS_alpha_vantage.csv
Saved news_cleaned_dataframe_NVDA_alpha_vantage.csv
Saved news_cleaned_dataframe_TSLA_alpha_vantage.csv
Saved news_cleaned_dataframe_AAPL_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_AMZN_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_BAC_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_GME_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_GS_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_NVDA_alpha_vantage_with_sentiment.csv
Saved news_cleaned_dataframe_TSLA_alpha_vantage_with_sentiment.csv


### Markets Insider

In [59]:
# Let's check the columns for the AAPL_market_vantage dataframe
news_dict['AAPL_markets_insider'].head(2)

,publish_date,title,source
0,4/4/2025 9:14:54 PM,"AAPL, JPM, GM: U.S. Stock Market Has Lost 11 T...",Markets Insider
1,4/4/2025 5:49:29 PM,"Earnings Could Drop by 28, Says Needham about ...",Markets Insider


In [7]:
# Extracting data from Markets Insider
data_market = cl.extract_markets_insider(news_dict)

In [61]:
# Lets check the columns for the AAPL_markets_insider dataframe
data_market['AAPL'].head(2)

,date,title,source
0,4/4/2025 9:14:54 PM,"AAPL, JPM, GM: U.S. Stock Market Has Lost 11 T...",Markets Insider
1,4/4/2025 5:49:29 PM,"Earnings Could Drop by 28, Says Needham about ...",Markets Insider


In [8]:
# Standardize the date format for the market data
data_market = cl.standardize_datetime_format(data_market)

In [9]:
# Lets check the columns for the AAPL_markets_insider dataframe
data_market['AAPL'].head(2)

,date,title,source
0,2025-04-04 21:14:54,"AAPL, JPM, GM: U.S. Stock Market Has Lost 11 T...",Markets Insider
1,2025-04-04 17:49:29,"Earnings Could Drop by 28, Says Needham about ...",Markets Insider


In [10]:
# Format date columnt 
dataframes_cleaned_markets_insider = cl.convert_date(data_market)

Stock: AAPL
First date: 2019-12-11
Last date: 2025-04-04
------------------------------
Stock: AMZN
First date: 2020-09-25
Last date: 2025-04-04
------------------------------
Stock: BAC
First date: 2005-03-28
Last date: 2025-04-04
------------------------------
Stock: GME
First date: 2011-06-28
Last date: 2025-04-04
------------------------------
Stock: GS
First date: 2011-05-20
Last date: 2025-04-04
------------------------------
Stock: NVDA
First date: 2014-01-06
Last date: 2025-04-04
------------------------------
Stock: TSLA
First date: 2022-04-01
Last date: 2025-04-04
------------------------------


In [65]:
# Lets check the columns for the AAPL_markets_insider
dataframes_cleaned_markets_insider['AAPL'].head(2)

,date,title,source
0,2025-04-04,"AAPL, JPM, GM: U.S. Stock Market Has Lost 11 T...",Markets Insider
1,2025-04-04,"Earnings Could Drop by 28, Says Needham about ...",Markets Insider


In [134]:
# Saving the data frames
cl.save_cleaned_dataframes(dataframes_cleaned_markets_insider, "markets_insider_news", data_folder_cleaned, prefix_word="news")

Saved news_cleaned_dataframe_AAPL_markets_insider_news.csv
Saved news_cleaned_dataframe_AMZN_markets_insider_news.csv
Saved news_cleaned_dataframe_BAC_markets_insider_news.csv
Saved news_cleaned_dataframe_GME_markets_insider_news.csv
Saved news_cleaned_dataframe_GS_markets_insider_news.csv
Saved news_cleaned_dataframe_NVDA_markets_insider_news.csv
Saved news_cleaned_dataframe_TSLA_markets_insider_news.csv


### Google search

In [11]:
# Let's check the columns for the AAPL_google_search dataframe
news_dict['AAPL_google_search'].head(2)

,publish_date,title,source
0,2022-04-06,"For 'Severance' star Adam Scott, there's no se...",Google News
1,2022-04-06,"Youre Still Being Tracked on the Internet, Jus...",Google News


In [12]:
# Extracting data from Google Search
data_google = cl.extract_google_search(news_dict)

In [13]:
# Lets check the columns for the AAPL_google_search dataframe
data_google['AAPL'].head(2)

,date,title,source
0,2022-04-06,"For 'Severance' star Adam Scott, there's no se...",Google News
1,2022-04-06,"Youre Still Being Tracked on the Internet, Jus...",Google News


In [14]:
# Convert the date column to datetime format 
dataframes_cleaned_google_search = cl.convert_date(data_google)

Stock: AMZN
First date: 2022-04-06
Last date: 2025-04-05
------------------------------
Stock: AAPL
First date: 2022-04-06
Last date: 2025-04-05
------------------------------
Stock: BAC
First date: 2022-04-06
Last date: 2025-04-05
------------------------------
Stock: GME
First date: 2022-04-08
Last date: 2025-04-05
------------------------------
Stock: GS
First date: 2022-04-06
Last date: 2025-04-05
------------------------------
Stock: NVDA
First date: 2022-04-06
Last date: 2025-04-05
------------------------------
Stock: TSLA
First date: 2022-04-06
Last date: 2025-04-05
------------------------------


In [15]:
# Lets check the columns for the AAPL_google_search
dataframes_cleaned_google_search['AAPL'].head(2)

,date,title,source
0,2022-04-06,"For 'Severance' star Adam Scott, there's no se...",Google News
1,2022-04-06,"Youre Still Being Tracked on the Internet, Jus...",Google News


In [72]:
# Saving the data frames
cl.save_cleaned_dataframes(dataframes_cleaned_google_search, "google_search_news", data_folder_cleaned, prefix_word="news")

Saved news_cleaned_dataframe_AMZN_google_search_news.csv
Saved news_cleaned_dataframe_AAPL_google_search_news.csv
Saved news_cleaned_dataframe_BAC_google_search_news.csv
Saved news_cleaned_dataframe_GME_google_search_news.csv
Saved news_cleaned_dataframe_GS_google_search_news.csv
Saved news_cleaned_dataframe_NVDA_google_search_news.csv
Saved news_cleaned_dataframe_TSLA_google_search_news.csv


### Merging news

In [16]:
# Merge all the news dataframes into one dataframe
news_combined = cl.concat_and_sort_dataframes_by_key([dataframes_cleaned_alpha_vantage, dataframes_cleaned_markets_insider, dataframes_cleaned_google_search])

In [17]:
stock_symbols = ["TSLA", "AAPL", "AMZN", "NVDA", "GS", "BAC", "GME"]

In [18]:
#Lets check the new dataframe for TSLA
ck.check(news_combined[stock_symbols[0]])

Number of columns: 3 and rows: 23134

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1102
title     22328
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11500
Google News        10959
Alpha Vantage        675
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
30

Count of duplicated values in subset:
34


In [19]:
#Lets check the new dataframe for AAPL
ck.check(news_combined[stock_symbols[1]])

Number of columns: 3 and rows: 23132

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1809
title     22046
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11500
Google News        10960
Alpha Vantage        672
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
106

Count of duplicated values in subset:
107


In [20]:
#Lets check the new dataframe for AMZN
ck.check(news_combined[stock_symbols[2]])

Number of columns: 3 and rows: 23149

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1619
title     21876
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11500
Google News        10959
Alpha Vantage        690
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
42

Count of duplicated values in subset:
52


In [21]:
#Lets check the new dataframe for NVDA
ck.check(news_combined[stock_symbols[3]])

Number of columns: 3 and rows: 22266

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2055
title     20730
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11195
Google News        10389
Alpha Vantage        682
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
89

Count of duplicated values in subset:
126


In [22]:
#Lets check the new dataframe for GS
ck.check(news_combined[stock_symbols[4]])

Number of columns: 3 and rows: 15877

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2341
title     14142
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Google News        10134
Markets Insider     5060
Alpha Vantage        683
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
171

Count of duplicated values in subset:
180


In [23]:
#Lets check the new dataframe for BAC
ck.check(news_combined[stock_symbols[5]])

Number of columns: 3 and rows: 16682

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2418
title     15102
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Google News        10941
Markets Insider     5035
Alpha Vantage        706
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
90

Count of duplicated values in subset:
100


In [24]:
#Lets check the new dataframe for GME
ck.check(news_combined[stock_symbols[6]])

Number of columns: 3 and rows: 6848

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date      1496
title     6186
source       3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    4193
Google News        1963
Alpha Vantage       692
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
11

Count of duplicated values in subset:
37


In [27]:
# Let's remove duplicates from the news_combined dataframe in date and title columns 
# since the different sources may have the same news
# and we want to keep only one of them.
news_combined = cl.remove_duplicates_subset(news_combined, ["date", "title"])

In [83]:
#Lets check the new dataframe for TSLA again after removing duplicates
ck.check(news_combined[stock_symbols[0]])

Number of columns: 3 and rows: 23100

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1102
title     22328
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11480
Google News        10950
Alpha Vantage        670
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [84]:
#Lets check the new dataframe for AAPL again after removing duplicates
ck.check(news_combined[stock_symbols[1]])

Number of columns: 3 and rows: 23025

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1809
title     22046
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11396
Google News        10959
Alpha Vantage        670
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [85]:
#Lets check the new dataframe for AMZN again after removing duplicates
ck.check(news_combined[stock_symbols[2]])

Number of columns: 3 and rows: 23097

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       1619
title     21876
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11462
Google News        10954
Alpha Vantage        681
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [86]:
#Lets check the new dataframe for NVDA again after removing duplicates
ck.check(news_combined[stock_symbols[3]])

Number of columns: 3 and rows: 22140

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2055
title     20730
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    11099
Google News        10368
Alpha Vantage        673
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [87]:
#Lets check the new dataframe for GS again after removing duplicates
ck.check(news_combined[stock_symbols[4]])

Number of columns: 3 and rows: 15697

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2341
title     14142
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Google News        10128
Markets Insider     4899
Alpha Vantage        670
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [88]:
#Lets check the new dataframe for BAC again after removing duplicates
ck.check(news_combined[stock_symbols[5]])

Number of columns: 3 and rows: 16582

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date       2418
title     15102
source        3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Google News        10938
Markets Insider     4945
Alpha Vantage        699
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [89]:
#Lets check the new dataframe for GME again after removing duplicates
ck.check(news_combined[stock_symbols[6]])

Number of columns: 3 and rows: 6811

Data types:
date      datetime64[ns]
title             object
source            object
dtype: object

Unique values count:
date      1496
title     6186
source       3
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['source'], dtype='object')

Unique value count for categorical columns:

source
Markets Insider    4173
Google News        1952
Alpha Vantage       686
Name: count, dtype: int64

Count of null values:
date      0
title     0
source    0
dtype: int64

Count of missing() values:
date      0
title     0
source    0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [90]:
# Saving the combined dataframe
cl.save_cleaned_dataframes(news_combined, "news", data_folder_cleaned, prefix_word="combined")

Saved combined_cleaned_dataframe_AAPL_news.csv
Saved combined_cleaned_dataframe_AMZN_news.csv
Saved combined_cleaned_dataframe_BAC_news.csv
Saved combined_cleaned_dataframe_GME_news.csv
Saved combined_cleaned_dataframe_GS_news.csv
Saved combined_cleaned_dataframe_NVDA_news.csv
Saved combined_cleaned_dataframe_TSLA_news.csv


### Fake news data

In [91]:
# Let's have a look to the fake news data
file_path = os.path.join(data_folder, 'training_data_fake_news.csv')
df_fake_news_training = cl.load_csv_with_column_names(file_path)
df_fake_news_training.head(2)


,bool_and_titles
0,0\tdonald trump sends out embarrassing new yea...
1,0\tdrunk bragging trump staffer started russia...


In [92]:
# Let's split the columns: one with the text and the other with the labels
df_fake_news_training = cl.split_bool_and_titles_column(df_fake_news_training)
df_fake_news_training.head(2)

,fake_news,title
0,0,donald trump sends out embarrassing new year‚s...
1,0,drunk bragging trump staffer started russian c...


In [93]:
# Let;s have a look to the fake news data for training
ck.check_fake(df_fake_news_training)

Number of columns: 2 and rows: 34152

Data types:
fake_news    object
title        object
dtype: object

Unique values count:
fake_news        2
title        32206
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['fake_news'], dtype='object')

Unique value count for categorical columns:

fake_news
0    17572
1    16580
Name: count, dtype: int64

Count of null values:
fake_news    0
title        0
dtype: int64

Count of missing() values:
fake_news    0
title        0
dtype: int64

Count of duplicated values:
1946


In [94]:
# Remove duplicates from the fake news training data
df_fake_news_training = cl.remove_duplicates(df_fake_news_training)

In [95]:
# Let's check the fake news training data after removing duplicates
ck.check_fake(df_fake_news_training)

Number of columns: 2 and rows: 32206

Data types:
fake_news    object
title        object
dtype: object

Unique values count:
fake_news        2
title        32206
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['fake_news'], dtype='object')

Unique value count for categorical columns:

fake_news
1    16181
0    16025
Name: count, dtype: int64

Count of null values:
fake_news    0
title        0
dtype: int64

Count of missing() values:
fake_news    0
title        0
dtype: int64

Count of duplicated values:
0


In [96]:
# Let's save the fake news data into a cleaned csv file
cl.save_dataframe_to_csv(df_fake_news_training, data_folder_cleaned, 'training_data_cleaned_fake_news.csv')

Saved DataFrame: training_data_cleaned_fake_news.csv


In [97]:
# Let's have a look to the fake news data for prediction
file_path = os.path.join(data_folder, 'testing_data_fake_news.csv')
df_fake_news_testing = cl.load_csv_with_column_names(file_path)
df_fake_news_testing.head(2)

,bool_and_titles
0,2\tcopycat muslim terrorist arrested with assa...
1,2\twow! chicago protester caught on camera adm...


In [98]:
# Let's split the columns: one with the text and the other with the labels
df_fake_news_testing = cl.split_bool_and_titles_column(df_fake_news_testing)
df_fake_news_testing.head(2)

,fake_news,title
0,2,copycat muslim terrorist arrested with assault...
1,2,wow! chicago protester caught on camera admits...


In [99]:
# Let's have a look to the fake news data for prediction
ck.check_fake(df_fake_news_testing)

Number of columns: 2 and rows: 9984

Data types:
fake_news    object
title        object
dtype: object

Unique values count:
fake_news       2
title        9208
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['fake_news'], dtype='object')

Unique value count for categorical columns:

fake_news
2     9982
﻿0       2
Name: count, dtype: int64

Count of null values:
fake_news    0
title        0
dtype: int64

Count of missing() values:
fake_news    0
title        0
dtype: int64

Count of duplicated values:
775


In [100]:
# Remove duplicates from the fake news training data
df_fake_news_testing = cl.remove_duplicates(df_fake_news_testing)

In [101]:
# Let's have a look to the fake news data for prediction after removing duplicates
ck.check_fake(df_fake_news_testing)

Number of columns: 2 and rows: 9209

Data types:
fake_news    object
title        object
dtype: object

Unique values count:
fake_news       2
title        9208
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index(['fake_news'], dtype='object')

Unique value count for categorical columns:

fake_news
2     9207
﻿0       2
Name: count, dtype: int64

Count of null values:
fake_news    0
title        0
dtype: int64

Count of missing() values:
fake_news    0
title        0
dtype: int64

Count of duplicated values:
0


In [102]:
# Let's save the fake news data into a cleaned csv file
cl.save_dataframe_to_csv(df_fake_news_testing, data_folder_cleaned, 'testing_data_cleaned_fake_news.csv')

Saved DataFrame: testing_data_cleaned_fake_news.csv


## Stocks

### Alpha Vantage

In [103]:
# Let's check the columns for the AAPL_alpha_vantage dataframe
stocks_dict['AAPL_alpha_vantage'].head(2)

,date,1. open,2. high,3. low,4. close,5. volume
0,2025-04-04,193.89,199.88,187.34,188.38,125910913
1,2025-04-03,205.54,207.49,201.25,203.19,103419006


In [104]:
# Lets clean the columns names
for key in stocks_dict.keys():
    stocks_dict[key] = cl.clean_column_names(stocks_dict[key])

In [105]:
# Lets rename the columns to the ticker name
stocks_dict = cl.rename_keys_to_ticker(stocks_dict)

In [106]:
# Let's check the columns for the AAP dataframe
stocks_dict['AAPL'].head(2)

,date,open,high,low,close,volume
0,2025-04-04,193.89,199.88,187.34,188.38,125910913
1,2025-04-03,205.54,207.49,201.25,203.19,103419006


In [107]:
# Set date in correct format 
dataframes_cleaned_stock = cl.convert_date(stocks_dict)

Stock: AAPL
First date: 1999-11-01
Last date: 2025-04-04
------------------------------
Stock: AMZN
First date: 1999-11-01
Last date: 2025-04-04
------------------------------
Stock: BAC
First date: 1999-11-01
Last date: 2025-04-04
------------------------------
Stock: GME
First date: 2002-02-13
Last date: 2025-04-04
------------------------------
Stock: GS
First date: 1999-11-01
Last date: 2025-04-04
------------------------------
Stock: NVDA
First date: 1999-11-01
Last date: 2025-04-04
------------------------------
Stock: TSLA
First date: 2010-06-29
Last date: 2025-04-04
------------------------------


In [108]:
# Lets check the columns for the AAPL
dataframes_cleaned_stock['AAPL'].head(2)

,date,open,high,low,close,volume
0,2025-04-04,193.89,199.88,187.34,188.38,125910913
1,2025-04-03,205.54,207.49,201.25,203.19,103419006


In [109]:
# Saving the data frames
cl.save_cleaned_dataframes(dataframes_cleaned_stock, "alpha_vantage", data_folder_cleaned, prefix_word="stocks")

Saved stocks_cleaned_dataframe_AAPL_alpha_vantage.csv
Saved stocks_cleaned_dataframe_AMZN_alpha_vantage.csv
Saved stocks_cleaned_dataframe_BAC_alpha_vantage.csv
Saved stocks_cleaned_dataframe_GME_alpha_vantage.csv
Saved stocks_cleaned_dataframe_GS_alpha_vantage.csv
Saved stocks_cleaned_dataframe_NVDA_alpha_vantage.csv
Saved stocks_cleaned_dataframe_TSLA_alpha_vantage.csv


### Yahoo

In [29]:
# Let's check the columns for the stock_AAPL_yahoo dataframe
stocks_dict_yahoo['stock_AAPL_yahoo'].head(2)

,Date,Close,High,Low,Open,Volume
0,NaN,AAPL,AAPL,AAPL,AAPL,AAPL
1,2022-05-09,149.85523986816406,153.5705818622736,149.29351226682695,152.68362217522588,131577900


In [30]:
# Lets clean the columns
for key in stocks_dict_yahoo.keys():
    stocks_dict_yahoo[key] = cl.remove_first_row(stocks_dict_yahoo[key])

In [31]:
# Lets clean the columns names
for key in stocks_dict_yahoo.keys():
    stocks_dict_yahoo[key] = cl.snake(stocks_dict_yahoo[key])

In [32]:
stocks_dict_yahoo['stock_AAPL_yahoo'].head(2)

,date,close,high,low,open,volume
0,2022-05-09,149.85523986816406,153.5705818622736,149.29351226682695,152.68362217522588,131577900
1,2022-05-10,152.26968383789062,154.46736088090017,150.71259117184084,153.26504901265346,115366700


In [33]:
# Lets rename the columns to the ticker name
stocks_dict_yahoo = cl.rename_keys_to_ticker(stocks_dict_yahoo, pos =1)

In [34]:
# Set date in correct format 
dataframes_cleaned_stock_yahoo = cl.convert_date(stocks_dict_yahoo)

Stock: AAPL
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: AMZN
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: BAC
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: GME
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: GS
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: NVDA
First date: 2022-05-09
Last date: 2025-05-07
------------------------------
Stock: TSLA
First date: 2022-05-09
Last date: 2025-05-07
------------------------------


In [37]:
dataframes_cleaned_stock_yahoo['AAPL'].head(2)

,date,close,high,low,open,volume
0,2022-05-09,149.85523986816406,153.5705818622736,149.29351226682695,152.68362217522588,131577900
1,2022-05-10,152.26968383789062,154.46736088090017,150.71259117184084,153.26504901265346,115366700


In [116]:
# Saving the data frames
cl.save_cleaned_dataframes(dataframes_cleaned_stock_yahoo, "yahoo", data_folder_cleaned, prefix_word="stocks")

Saved stocks_cleaned_dataframe_AAPL_yahoo.csv
Saved stocks_cleaned_dataframe_AMZN_yahoo.csv
Saved stocks_cleaned_dataframe_BAC_yahoo.csv
Saved stocks_cleaned_dataframe_GME_yahoo.csv
Saved stocks_cleaned_dataframe_GS_yahoo.csv
Saved stocks_cleaned_dataframe_NVDA_yahoo.csv
Saved stocks_cleaned_dataframe_TSLA_yahoo.csv


### Merging news and stocks

In [144]:
# Merging same days new per stock
processed_news = cl.process_news_data(news_combined)

# Merging the stocks dataframes with the news dataframes
merged_dict =  cl.merge_stock_and_news(dataframes_cleaned_stock_yahoo, processed_news)

In [145]:
merged_dict['AAPL'].head(2)

,date,close,high,low,open,volume,title
0,2022-05-09,149.85523986816406,153.5705818622736,149.29351226682695,152.68362217522588,131577900,4 BlueChip Stocks to Buy for May 2022 Where to...
1,2022-05-10,152.26968383789062,154.46736088090017,150.71259117184084,153.26504901265346,115366700,Twitter Stock Has Risks After Deal With Elon M...


In [119]:
ck.check(merged_dict['AAPL'])

Number of columns: 7 and rows: 752

Data types:
date      datetime64[ns]
close             object
high              object
low               object
open              object
volume            object
title             object
dtype: object

Unique values count:
date      752
close     744
high      752
low       752
open      752
volume    752
title     730
dtype: int64

These columns appear to be categorical (less than 20 unique values):
Index([], dtype='object')

Unique value count for categorical columns:

Count of null values:
date       0
close      0
high       0
low        0
open       0
volume     0
title     22
dtype: int64

Count of missing() values:
date      0
close     0
high      0
low       0
open      0
volume    0
title     0
dtype: int64

Count of duplicated values:
0

Count of duplicated values in subset:
0


In [121]:
merged_dict['AAPL'][merged_dict['AAPL']['title'].isna()].head(22)

,date,close,high,low,open,volume,title
730,2025-04-07,181.4600067138672,194.14999389648438,174.6199951171875,177.1999969482422,160466300,NaN
731,2025-04-08,172.4199981689453,190.33999633789062,169.2100067138672,186.6999969482422,120859500,NaN
732,2025-04-09,198.85000610351562,200.61000061035156,171.88999938964844,171.9499969482422,184395900,NaN
733,2025-04-10,190.4199981689453,194.77999877929688,183.0,189.07000732421875,121880000,NaN
734,2025-04-11,198.14999389648438,199.5399932861328,186.05999755859375,186.10000610351562,87435900,NaN
735,2025-04-14,202.52000427246094,212.94000244140625,201.16000366210938,211.44000244140625,101352900,NaN
736,2025-04-15,202.13999938964844,203.50999450683594,199.8000030517578,201.86000061035156,51343900,NaN
737,2025-04-16,194.27000427246094,200.6999969482422,192.3699951171875,198.36000061035156,59732400,NaN
738,2025-04-17,196.97999572753906,198.8300018310547,194.4199981689453,197.1999969482422,51334300,NaN
739,2025-04-21,193.16000366210938,193.8000030517578,189.80999755859375,193.27000427246094,46742500,NaN


In [146]:
# The last dates are not include in the news dataframes so result in NaN values, for the analysis we can drop them
for key in merged_dict.keys():
    merged_dict[key] = merged_dict[key].dropna(subset=['title'])
    merged_dict[key] = merged_dict[key].reset_index(drop=True)

In [147]:
# Saving the data frames
cl.save_cleaned_dataframes(merged_dict, "merged", data_folder_cleaned, prefix_word="stocks_news")

Saved stocks_news_cleaned_dataframe_AAPL_merged.csv
Saved stocks_news_cleaned_dataframe_AMZN_merged.csv
Saved stocks_news_cleaned_dataframe_BAC_merged.csv
Saved stocks_news_cleaned_dataframe_GME_merged.csv
Saved stocks_news_cleaned_dataframe_GS_merged.csv
Saved stocks_news_cleaned_dataframe_NVDA_merged.csv
Saved stocks_news_cleaned_dataframe_TSLA_merged.csv
